In [ ]:
import pandas as pd
import datetime
import unidecode
import re
from bs4 import BeautifulSoup
import requests
import os
import glob

In [ ]:
lst_arquivo = glob.glob(os.path.join(r'data\datasus','*.csv'))
data = pd.concat(pd.read_csv(i) for i in lst_arquivo)

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
data.head()

In [ ]:
data_dep= data[['CAUSABAS',
               'DTOBITO','DTNASC',
               'SEXO','RACACOR',
               'ESTCIV','ESC',
               'LOCOCOR','CODMUNOCOR',
               'GESTACAO','PESO',
               'CIRCOBITO','CAUSABAS_O']]

In [ ]:
data_dep.shape

In [ ]:
# data_dep= data[(data.CAUSABAS.str.startswith('X84')) | (data.CAUSABAS.str.startswith('X60'))][['CAUSABAS',
#                                                                                                'DTOBITO','DTNASC',
#                                                                                                'SEXO','RACACOR',
#                                                                                                'ESTCIV','ESC',
#                                                                                                'LOCOCOR','CODMUNOCOR',
#                                                                                                'GESTACAO','PESO',
#                                                                                                'CIRCOBITO','CAUSABAS_O']]

In [ ]:
data_dep['DTOBITO'] = data_dep.DTOBITO.astype(str)
data_dep['DTNASC'] = data_dep.DTNASC.astype(str)

In [ ]:
data_mun = pd.read_csv(r'data\tabmun.csv')

In [ ]:
data_mun.CODMUN = data_mun.CODMUN.astype(str)
data_mun['CODMUN6'] = data_mun.CODMUN.str[0:6]
data_mun.CODMUN6 = data_mun.CODMUN6.astype(int)

In [ ]:
data_dep = pd.merge(data_dep, data_mun, left_on='CODMUNOCOR', right_on='CODMUN6')

In [ ]:
def tratar_racacor(racacor):
    if racacor == 1:
        return 'BRANCA'
    elif racacor == 2:
        return 'PRETA'
    elif racacor == 3:
        return 'PRETA'
    elif racacor == 4:
        return 'PARDA'
    elif racacor == 5:
        return 'INDIGENA'
    else:
        return 'NA'

In [ ]:
def tratar_estadocivil(civil):
    if civil == 1:
        return 'SOLTEIRO'
    elif civil == 2:
        return 'CASADO'
    elif civil == 3:
        return 'VIUVO'
    elif civil == 4:
        return 'SEPARADO'
    elif civil == 5:
        return 'UNIAO CONSENSUAL'
    else:
        return 'NA'

In [ ]:
def tratar_sexo(sexo):
    if sexo == 1:
        return 'MASCULINO'
    elif sexo == 2:
        return 'FEMININO'
    else:
        return 'NA'

In [ ]:
def tratar_obito(obito):
    if obito == 1:
        return 'ACIDENTE'
    elif obito == 2:
        return 'SUICIDIO'
    elif obito == 3:
        return 'HOMICIDIO'
    elif obito == 4:
        return 'OUTRO'
    else:
        return 'NA'

In [ ]:
def tratar_idade(series):
    data_obito = str(series['DTOBITO'])
    data_nasci = str(series['DTNASC'])
    data_nasci = data_nasci.replace('.0','').replace('nan','')
    if len(data_obito) < 8:
        data_obito = '0' + data_obito
    
    if len(data_nasci) < 8 and len(data_nasci) > 0:
        data_nasci = '0' + data_nasci
    
    if data_nasci != '' and data_nasci != 0:
        dias = datetime.datetime.strptime(data_obito, '%d%m%Y').year - datetime.datetime.strptime(data_nasci, '%d%m%Y').year
    else:
        dias = None
    return dias

In [ ]:
# def tratar_causa(causa):
#     '''CID - Classificação Internacional de Doenças
#         CID 10 - X84   	Lesão autoprovocada intencionalmente por meios não especificados
#         CID 10 - X60   	Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos
#     '''
#     if causa.startswith('X84'):
#         return 'Lesão autoprovocada intencionalmente por meios não especificados'
#     else:
#         return 'Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos'

In [ ]:
data_dep['NMRACACOR'] = data_dep.RACACOR.apply(tratar_racacor)
data_dep['NMESTCIV'] = data_dep.ESTCIV.apply(tratar_estadocivil)
data_dep['NMSEXO'] = data_dep.SEXO.apply(tratar_sexo)
data_dep['NMCIRCOBITO'] = data_dep.CIRCOBITO.apply(tratar_obito)
#data_dep['NMCAUSABAS_O'] = data_dep.CAUSABAS_O.apply(tratar_causa)
data_dep['IDADE'] = data_dep.apply(tratar_idade, axis=1)

In [ ]:
data_idhm = pd.read_csv(r'data\IDHM_2010_Brasil.csv',
                        encoding='latin-1',
                        header=1,
                        sep=';')

In [ ]:
data_idhm.Nome = data_idhm.Nome.apply(lambda x: unidecode.unidecode(x).upper())
data_idhm['UF'] = data_idhm.Nome.apply(lambda x: re.findall('\(.*?\)',x)[0].replace('(','').replace(')',''))
data_idhm.Nome = data_idhm.Nome.apply(lambda x: ''.join(re.findall('\w*[^(^\w\)]',x)).strip())
data_idhm['Posição'] = data_idhm['Posição'].apply(lambda x: x.replace('º','').strip())
data_idhm['Posição'] = data_idhm['Posição'].astype(int)

In [ ]:
data_dep = pd.merge(data_dep, data_idhm, left_on=['NOMMUN','ESTMUN'], right_on=['Nome','UF'])

In [ ]:
data_dep.shape

In [ ]:
url = 'http://www.medicinanet.com.br/categorias/lista_cid10.htm'
soup = BeautifulSoup(requests.get(url).text)

In [ ]:
lst_cid = [item.text.replace('\xa0',' ') for item in soup.find("div", {"id": "texto"}).findAll('a')]
dic_cid = {item.split('-')[0].strip().replace('.',''):item.split('-')[1].strip() for item in lst_cid}
data_cid = pd.DataFrame(list(dic_cid.items()), columns=['CODCID','NMCID'])

In [ ]:
data_cid.head()

In [ ]:
data_dep = pd.merge(data_dep, data_cid, left_on='CAUSABAS', right_on='CODCID')

In [ ]:
data_dep.shape

In [ ]:
data_dep.dtypes

In [ ]:
data_dep.head()

In [ ]:
#ren = lambda x:''.join(re.findall('[\w\s]',x)).replace(' ', '_').lower()
#admissions = admissions.rename(columns={item: ren(item) for item in admissions.columns})

data_dep = data_dep.rename(columns={'IDHM (2010)': 'IDHM',
                                     'IDHM Renda (2010)':'IDHM_RENDA',
                                     'IDHM Longevidade (2010)': 'IDHM_LONGEVIDADE',
                                     'IDHM Educação (2010)': 'IDHM_EDUCACAO',
                                     'Posição':'POSICAO',
                                   'Nome':'NOME'})
data_dep.columns

In [ ]:
data_dep.head(1).T

In [ ]:
data_dep = data_dep[['CAUSABAS', 'DTOBITO','CAUSABAS_O',
       'NOMMUN', 'NMRACACOR', 'NMESTCIV','NMCIRCOBITO',
       'NMSEXO', 'IDADE', 'POSICAO', 'NOME', 'IDHM','UF','CODCID',
       'NMCID']]

In [ ]:
data_dep.to_csv('data\datasus.csv')

In [ ]:
data_dep.head(5)

In [ ]:
data.columns

In [ ]:
data['DTNASC'].isnull().sum()